In [ ]:
import os
import numpy as np
from ultrack import MainConfig, load_config, track, to_tracks_layer, tracks_to_zarr
import zarr
import pandas as pd

In [ ]:
# # set parameters
root = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\killi_tracker\\"
project_name = "230425_EXP21_LCP1_D6_1pm_DextranStabWound"
image_zarr = os.path.join(root, "built_data", "zarr_image_files",  project_name + ".zarr")
# label_zarr = os.path.join(root, "built_data", "cleaned_cell_labels", project_name + ".zarr")
ds_factor = 1
config_name = "tracking_jordao_full.txt"
tracking_folder = config_name.replace(".txt", "")
tracking_folder = tracking_folder.replace(".toml", "")

figure_path = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\slides\\killifish\\20240430\\"

model_name = "LCP-Multiset-v1"
mask_path = os.path.join(root, "built_data", "stitched_labels", model_name, project_name + "_labels_stitched.zarr")
mask_zarr = zarr.open(mask_path, mode="r")

## Load tracks and image data

In [ ]:
cfg = load_config(os.path.join(root, "metadata", project_name, config_name))
tracks_df, graph = to_tracks_layer(cfg)

In [ ]:
data_tzyx = zarr.open(image_zarr, mode='r')

In [ ]:
from scipy.interpolate import interp1d

# make array to specify wound location
wound_center_array = np.asarray([[315, 457, 424], [315, 427, 424], [315, 405, 410], [315, 372, 434]])
center_frames = np.asarray([0, 59, 84, 118])
frame_vec_full = np.arange(data_tzyx.shape[0])
interpf = interp1d(center_frames, wound_center_array, axis=0)
wound_center_interp = interpf(frame_vec_full)

## Fit sphere for distance estimations

In [ ]:
# from src.utilities.functions import sphereFit, cart_to_sphere
# import pandas as pd
# model_name = "LCP-Multiset-v1"
# mask_path = os.path.join(root, "built_data", "stitched_labels", model_name, project_name + "_labels_stitched.zarr")
# mask_zarr = zarr.open(mask_path, mode="r")

# n_points_fit = 10000
# ref_frame = 59

# mask_array = mask_zarr[0]
# z_ref, y_ref, x_ref = np.meshgrid(range(mask_array.shape[0]),
#                                               range(mask_array.shape[1]),
#                                               range(mask_array.shape[2]),
#                                               indexing="ij")

# # get location of masked pixels
# nc_z = z_ref[mask_array > 0]
# nc_y = y_ref[mask_array > 0]
# nc_x = x_ref[mask_array > 0]

# np.random.seed(364)
# sphere_df_list = []
# for frame in tqdm(frame_vec_full, "Fitting spheres to lightsheet data..."):
#     # get points
#     mask_array = mask_zarr[frame]
#     # get location of masked pixels
#     nc_z = z_ref[mask_array > 0]
#     nc_y = y_ref[mask_array > 0]
#     nc_x = x_ref[mask_array > 0]
#     # choose indices to fit
#     fit_indices = np.random.choice(range(len(nc_z)), np.min([n_points_fit, len(nc_z)]), replace=False)
#     # fit to sphere
#     center_point = np.empty((3, ))
#     radius, center_point[2], center_point[1], center_point[0] = sphereFit(nc_x[fit_indices], nc_y[fit_indices], nc_z[fit_indices])
#     # store data
#     sphere_df = pd.DataFrame(center_point[np.newaxis, :], columns=["Z", "Y", "X"])
#     sphere_df["r"] = radius
#     sphere_df["t"] = frame
#     sphere_df["project"] = project_name
#     sphere_df_list.append(sphere_df)



In [ ]:
# sphere_df_list[8] = sphere_df_list[7]
# sphere_df_master = pd.concat(sphere_df_list, axis=0, ignore_index=True)
# sphere_df_master.loc[8, "t"] = 8
# sphere_df_master = sphere_df_master.drop_duplicates(subset=["t", "project"]).loc[:, ["t", "r", "Z", "Y", "X"]]
# sphere_df_master_sm = sphere_df_master.copy()
# sphere_df_master_sm.loc[:, ["Z", "Y", "X"]] = sphere_df_master_sm.loc[:, ["Z", "Y", "X"]].rolling(10, center=True, min_periods=1).mean()


sphere_df_master_sm = pd.read_csv(os.path.join(root, "metadata", project_name, "sphere_df.csv"))
sphere_df_master_sm.head()

## Calculate net velocities as a function of time and distance from the stab wound

In [ ]:
def find_surface_point(raw_point, center_point, radius):
    # find closest point that is on the sphere
    c_vec = raw_point - center_point
    c_vec = c_vec / np.sqrt(np.sum(c_vec**2))
    r_vec = c_vec * radius
    surf_point = center_point + r_vec

    return surf_point

In [ ]:
from tqdm import tqdm 
from astropy.coordinates import cartesian_to_spherical, spherical_to_cartesian

# set meanungful units
tres = 60
scale_vec = np.asarray([1.5, 1.5, 1.5])

time_window = 10 # all calculations will be based on N-frame track fragments 

# loop through time points and get list of distances and signed velocities
master_vel_vec = []

for f in tqdm(frame_vec_full):
    center_point = sphere_df_master_sm.loc[f, ["Z", "Y", "X"]]
    radius = sphere_df_master_sm.loc[f, "r"]
    
    # get wound location
    wound_center_raw = wound_center_interp[f]
    # find closest point that is on the sphere
    wound_center = find_surface_point(wound_center_raw, center_point, radius)

    # add wound center info 
    wc = wound_center - center_point
    w_sph = cartesian_to_spherical(wc[2], wc[1], wc[0])
    w_sph = np.asarray([c.value for c in w_sph])
    sphere_df_master_sm.loc[f, ["wound_z", "wound_y", "wound_x"]] = wound_center[:]
    sphere_df_master_sm.loc[f, ["wound_r", "wound_phi", "wound_theta"]] = w_sph[:]
    
    # find extant tracks
    f_start = f
    f_stop = f + time_window-1
    tracks_filtered = tracks_df.loc[(tracks_df["t"]==f_start) | (tracks_df["t"]==f_stop), :].reset_index(drop=True)
    
    # find tracks that are present for entire window
    tracks_u, tracks_c = np.unique(tracks_filtered["track_id"], return_counts=True)

    tracks_to_use = tracks_u[tracks_c==2]

    vel_vec_temp = []
    for i, id in enumerate(tracks_to_use):
        # get start and end locations
        tracks_temp = tracks_filtered.loc[tracks_filtered["track_id"]==id, :].reset_index(drop=True)
        zyx0_raw = tracks_temp.loc[0, ["z", "y", "x"]].to_numpy()
        zyx1_raw = tracks_temp.loc[1, ["z", "y", "x"]].to_numpy()
        
        # get closest points on sphere surface
        zyx0 = find_surface_point(zyx0_raw, center_point, radius)
        zyx1 = find_surface_point(zyx1_raw, center_point, radius)
        d_cart = zyx1-zyx0
        
        # calculate distance to wound site
        d0 = np.sqrt(np.sum((wound_center-zyx0)**2))#2*radius*np.arcsin(np.sqrt(np.sum((wound_center-zyx0)**2))/(2*radius))
        d1 = np.sqrt(np.sum((wound_center-zyx1)**2))#2*radius*np.arcsin(np.sqrt(np.sum((wound_center-zyx1)**2))/(2*radius))

        # get spherical coordinates
        zyx0_c = zyx0 - center_point
        zyx1_c = zyx1 - center_point
        
        sph0 = cartesian_to_spherical(zyx0_c[2], zyx0_c[1], zyx0_c[0])
        sph0 = np.asarray([c.value for c in sph0])
        sph1 = cartesian_to_spherical(zyx1_c[2], zyx1_c[1], zyx1_c[0])
        sph1 = np.asarray([c.value for c in sph1])
        sph_d = sph1 - sph0
        master_vel_vec.append([id, f_start, d0, (d1-d0), sph0[0], sph0[1], sph0[2], sph_d[0], sph_d[1], sph_d[2],
                              zyx0[0], zyx0[1], zyx0[2], d_cart[0], d_cart[1], d_cart[2]])


    # master_vel_vec.append(vel_vec_temp)
    

In [ ]:
tracks_df.reset_index(drop=True, inplace=True)

for f in tqdm(frame_vec_full):
    center_point = sphere_df_master_sm.loc[f, ["Z", "Y", "X"]]
    radius = sphere_df_master_sm.loc[f, "r"]
    
    # get wound location
    wound_center_raw = wound_center_interp[f]
    # find closest point that is on the sphere
    wound_center = find_surface_point(wound_center_raw, center_point, radius)

    # add wound center info 
    wc = wound_center - center_point
    w_sph = cartesian_to_spherical(wc[2], wc[1], wc[0])
    w_sph = np.asarray([c.value for c in w_sph])
    # sphere_df_master_sm.loc[f, ["wound_z", "wound_y", "wound_x"]] = wound_center[:]
    # sphere_df_master_sm.loc[f, ["wound_r", "wound_phi", "wound_theta"]] = w_sph[:]
    
    # find extant tracks
    f_start = f
    f_stop = f + time_window-1
    # tracks_filtered = tracks_df.loc[(tracks_df["t"]==f_start), :].reset_index(drop=True)
    f_indices = np.where(tracks_df["t"]==f_start)[0]
    # find tracks that are present for entire window
    # tracks_u, tracks_c = np.unique(tracks_filtered["track_id"], return_counts=True)

    # tracks_to_use = tracks_u#[tracks_c==2]

    # vel_vec_temp = []
    for index in f_indices:
        # get start and end locations
        # tracks_temp = tracks_filtered.loc[tracks_filtered["track_id"]==id, :].reset_index(drop=True)
        zyx0_raw = tracks_df.loc[index, ["z", "y", "x"]].to_numpy()
        # zyx1_raw = tracks_temp.loc[1, ["z", "y", "x"]].to_numpy()
        
        # get closest points on sphere surface
        zyx0 = find_surface_point(zyx0_raw, center_point, radius)
        # zyx1 = find_surface_point(zyx1_raw, center_point, radius)
        # d_cart = zyx1-zyx0
        
        # calculate distance to wound site
        d0 = np.sqrt(np.sum((wound_center-zyx0)**2))#2*radius*np.arcsin(np.sqrt(np.sum((wound_center-zyx0)**2))/(2*radius))
        tracks_df.loc[index, "distance"] = d0 / 1.5

In [ ]:
tracks_df.to_csv(os.path.join(root, "metadata", project_name, "tracks_df_dist.csv"), index=False)

In [ ]:
tracks_df.head()

In [ ]:
vel_df = pd.DataFrame(master_vel_vec, columns=["track_id", "frame", "d0", "net_distance", "r0", "phi0", "theta0", "r_d", "phi_d", "theta_d",
                                              "z0", "y0", "x0", "dz", "dy", "dx"])
vel_df.tail(5)

In [ ]:
# import plotly.figure_factory as ff
# import plotly.express as px
# import plotly.graph_objects as go

# frames=np.asarray([10, 20, 30, 40, 50])
# ft = np.isin(vel_df["frame"],frames)
# fig = ff.create_quiver(x=vel_df.loc[ft, "x0"], y=vel_df.loc[ft, "y0"], u=vel_df.loc[ft, "dx"], v=vel_df.loc[ft, "dy"],
#                       marker_size=12, scale=1)

# fig.add_trace(go.Scatter(x=[wound_center_interp[frame, 2]], 
#                          y=[wound_center_interp[frame, 1]], mode="markers", marker=dict(size=10)))

# fig.update_layout(
#     autosize=False,
#     width=500,
#     height=500)

# fig.show()

## Do the great circle distances make sense?

In [ ]:
for f in frame_vec_full:
    vel_df.loc[vel_df["frame"]==f,["wound_z", "wound_y", "wound_x"]] = np.tile(wound_center_interp[f, :], (np.sum(vel_df["frame"].to_numpy()==f), 1))

vel_df["x_rel"] = vel_df["x0"] - vel_df["wound_x"]
vel_df["y_rel"] = vel_df["y0"] - vel_df["wound_y"]
vel_df["t_polar"] = np.arctan2(vel_df["y_rel"], vel_df["x_rel"])
vel_df["x_polar"] = np.multiply(vel_df["d0"], np.cos(vel_df["t_polar"]))
vel_df["y_polar"] = np.multiply(vel_df["d0"], np.sin(vel_df["t_polar"]))
vel_df["u_polar"] = np.divide(np.multiply(vel_df["net_distance"], vel_df["x_polar"]), vel_df["d0"])
vel_df["v_polar"] = np.divide(np.multiply(vel_df["net_distance"], vel_df["y_polar"]), vel_df["d0"])

vel_df["frame_cat"] = vel_df["frame"].astype(str)
vel_df["net_vel"] = -vel_df["net_distance"] * 1.5 / time_window

vel_df.to_csv(os.path.join(root, "metadata", project_name, "velocity_df.csv"), index=False)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# from matplotlib.colors import Normalize

# frames=np.arange(20, 50) #np.asarray([90, 95])

# ft = np.isin(vel_df["frame"],frames)

# vel_df["net_dist_norm"] = np.divide(vel_df["net_distance"], np.sqrt(vel_df["dx"]**2+vel_df["dy"]**2+vel_df["dz"]**2))

# x=vel_df.loc[ft, "x_polar"]
# y=vel_df.loc[ft, "y_polar"] 
# u=vel_df.loc[ft, "u_polar"]
# v=vel_df.loc[ft, "v_polar"]


# colors = vel_df.loc[ft, "net_dist_norm"].to_numpy()
# colors[np.isnan(colors)] = 0
# colors = np.clip(colors,-clip_val, clip_val)
# # ft = colors < 0
# colors = colors - np.min(colors)
# colors = colors / np.max(colors)

# colormap = cm.RdYlBu


# # pick your colormap here, refer to 
# # http://matplotlib.org/examples/color/colormaps_reference.html
# # and
# # http://matplotlib.org/users/colormaps.html
# # for details
# plt.figure(figsize=(6, 6))
# # plt.xlim(-2, 2)
# # plt.ylim(-2, 2)
# plt.quiver(x, y, u, v, color=colormap(colors), scale=0.75, angles='xy',scale_units='xy')
#            #scale_units='xy', scale=1, pivot='mid')

# plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

frames=np.arange(50, 62, 2) # np.asarray([20, 30,  40, 50, 60, 70])
vel_df["net_vel"] = -vel_df["net_distance"] * 1.5 / time_window
ft = np.isin(vel_df["frame"],frames)

fig = px.scatter(vel_df.loc[ft], x="d0", y="net_vel", facet_col="frame",  trendline="lowess", opacity=0.1, 
                labels={"frame":"minutes", "net_vel": "net velocity (um/s)", "d0":"wound dist (um)"})
fig.update_yaxes(range=[-5, 5])               
# fig.update_layout(xaxis_title="distance from wound (um)", yaxis_title="net velocity toward wound (um/minute)")
fig.show()


In [ ]:
def get_n_colors_from_colormap(colorscale_name, n_colors):
    # Get a Plotly colorscale by name
    colorscale = px.colors.get_colorscale(colorscale_name)

    sample_points = np.linspace(0,1,n_colors)
    # Sample n_colors from the colorscale
    sampled_colors = px.colors.sample_colorscale(colorscale, samplepoints=list(sample_points))
    
    return sampled_colors

# Example usage
colors = get_n_colors_from_colormap('Viridis', 15)
print(colors)

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

cmap = "PiYG"
frames=np.arange(30, 90, 10)

c_list = get_n_colors_from_colormap(cmap, len(frames))

ft = np.isin(vel_df["frame"],frames)

fig = px.scatter(vel_df.loc[ft], x="d0", y="net_vel", color="frame_cat",  trendline="lowess", opacity=0, color_discrete_sequence=c_list, 
                labels={"frame":"minutes", "frame_cat": "minutes", "net_vel": "net velocity (um/s)", "d0":"wound dist (um)"})
fig.update_yaxes(range=[-3, 3])  
fig.update_xaxes(range=[50, 500])  

iter = 0
for i in range(1,2*len(frames),2):
    fig.data[i].update(line_color=c_list[iter], line_width=5)
    iter+=1

fig.update_layout(
    autosize=False,
    width=500,
    height=500)

fig.write_image(figure_path + "net_vel_trends.png", scale=4)
fig.show()

In [ ]:
a = list(range(1,2*len(frames),2))
a

In [ ]:
vel_df["net_vel_total"] = np.sqrt(vel_df["dx"]**2+vel_df["dy"]**2+vel_df["dz"]**2)* 1.5 / time_window

frames=np.asarray([10, 20, 30,  40, 50, 60, 70])
ft = np.isin(vel_df["frame"],frames)

fig = px.scatter(vel_df.loc[ft], x="d0", y="net_vel_total", facet_col="frame",  trendline="lowess", opacity=0.1, 
                labels={"frame":"minutes", "net_vel_total": "net velocity (um/s)", "d0":"wound dist (um)"})
fig.update_yaxes(range=[0, 10])               
# fig.update_layout(xaxis_title="distance from wound (um)", yaxis_title="net velocity toward wound (um/minute)")
fig.show()

In [ ]:
fig = px.scatter_3d(tracks_df, x="x", y="y", z="z", color="track_id", opacity=0.1)
fig.add_trace(go.Scatter3d(x=wound_center_interp[:, 2], y=wound_center_interp[:, 1], z=wound_center_interp[:, 0], mode="markers"))
fig.show()

In [ ]:

vel_array = np.asarray(master_vel_vec)
plot_filter = (vel_array[:, 0] >= 10) & (vel_array[:, 0] <= 30)

plt.scatter(x=vel_array[plot_filter, 1], y=vel_array[plot_filter, 2])
plt.show()